In [76]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

### Games and Outcomes

In [77]:
import csv
global players
players = 0
player_dict = dict()
with open('train.csv', newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if str(row[1]) not in player_dict.values():
            player_dict[players] = str(row[1])
            players += 1
print(player_dict)

{0: 'MC', 1: 'Life', 2: 'sOs', 3: 'Polt', 4: 'TY', 5: 'INnoVation', 6: 'Mvp', 7: 'ByuN', 8: 'MMA', 9: 'PartinG', 10: 'Snute', 11: 'Zest', 12: 'Maru', 13: 'Nerchio', 14: 'TaeJa', 15: 'NesTea', 16: 'Stephano', 17: 'Dark', 18: 'Neeb', 19: 'HerO', 20: 'Leenock', 21: 'Bomber', 22: 'DongRaeGu', 23: 'Stats', 24: 'HyuN', 25: 'Rain', 26: 'Solar', 27: 'Jaedong', 28: 'herO', 29: 'Classic', 30: 'MarineKing', 31: 'Scarlett', 32: 'aLive', 33: 'soO', 34: 'NaNiwa', 35: 'viOLet', 36: 'ShoWTimE', 37: 'MaNa', 38: 'Sen', 39: 'ForGG', 40: 'Dear', 41: 'Soulkey', 42: 'PuMa', 43: 'GuMiho', 44: 'HuK', 45: 'Hydra', 46: 'SpeCial', 47: 'TRUE', 48: 'Jim', 49: 'Elazer', 50: 'jjakji', 51: 'XiGua', 52: 'MacSed', 53: 'Patience', 54: 'StarDust', 55: 'San', 56: 'TooDming', 57: 'VortiX', 58: 'FruitDealer', 59: 'Creator', 60: 'YoDa', 61: 'ByuL', 62: 'ThorZaIN', 63: 'iAsonu', 64: 'Trap', 65: 'Rogue', 66: 'TLO', 67: 'Kelazhur', 68: 'Losira', 69: 'uThermal', 70: 'Symbol', 71: 'Genius', 72: 'Bly', 73: 'Lilbow', 74: 'Bunny', 7

In [78]:
def get_id(val):
    for key, value in player_dict.items():
        if val == value:
             return key

games = []
with open('train.csv', newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        player1 = get_id(str(row[1]))
        player2 = get_id(str(row[4]))
        if player2 is None:
            global players
            player_dict[players] = str(row[4])
            player2 = players
            players += 1
        if row[2] == "[loser]":
            score = -1
        else:
            score = +1
        games.append((player1, player2, score))
print(games[0:20]) #it works

[(0, 23, -1), (0, 17, -1), (0, 5, -1), (0, 47, -1), (0, 136, 1), (0, 32, 1), (0, 86, 1), (0, 11, -1), (0, 43, -1), (0, 11, 1), (0, 43, -1), (0, 136, -1), (0, 11, 1), (0, 342, -1), (0, 294, 1), (0, 108, 1), (0, 135, 1), (0, 70, -1), (0, 51, 1), (0, 527, 1)]


### Win probability and graphical model

In [79]:
nplayers = max( [max(g[0],g[1]) for g in games] )+1
nlevels = 10   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability

# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in games:
    P1,P2,win = g[0],g[1],g[2]
    if P1 != P2:
        if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
        inside = gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin)
        factors.append(inside)

In [ ]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [ ]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    #lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

### Ranking players by predicted skill

In [ ]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

### Predicting match outcomes